In [1]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import pandas as pd
import numpy as np

# 전처리
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
import random

# 모델
from lightgbm import LGBMClassifier

# 시각화 툴
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [3]:
# data load

inputs = pd.read_csv('./content/train.csv')
labels = pd.read_csv('./content/train_label.csv')
test = pd.read_csv('./content/test.csv')

In [4]:
time_series_length = pd.concat([inputs['EMAIL'].value_counts()])

In [5]:
shortest_length = time_series_length[-1]

In [6]:
shortest_length

35

In [7]:
arranged_labels = []
for id in inputs['EMAIL'].unique():
            idx = inputs['EMAIL'][inputs['EMAIL'] == id].index
            start_idx = idx[0]
            end_idx = idx[-1]
            inputs.drop(list((range(start_idx + shortest_length , end_idx+1))), axis=0, inplace=True)
            inputs = inputs.reset_index(drop=True)
            label_idx = labels['SAMPLE_EMAIL'][labels['SAMPLE_EMAIL'] == id].index[0]
            arranged_labels.append(labels['DIAG_NM'][label_idx])

In [8]:
arranged_labels

['CN',
 'MCI',
 'MCI',
 'CN',
 'MCI',
 'CN',
 'MCI',
 'CN',
 'MCI',
 'CN',
 'CN',
 'MCI',
 'Dem',
 'CN',
 'CN',
 'CN',
 'CN',
 'MCI',
 'MCI',
 'CN',
 'MCI',
 'CN',
 'CN',
 'CN',
 'MCI',
 'Dem',
 'CN',
 'Dem',
 'CN',
 'MCI',
 'CN',
 'MCI',
 'MCI',
 'CN',
 'CN',
 'MCI',
 'CN',
 'CN',
 'CN',
 'CN',
 'CN',
 'CN',
 'CN',
 'CN',
 'CN',
 'CN',
 'CN',
 'CN',
 'CN',
 'MCI',
 'CN',
 'CN',
 'CN',
 'MCI',
 'MCI',
 'CN',
 'CN',
 'MCI',
 'CN',
 'CN',
 'CN',
 'CN',
 'Dem',
 'MCI',
 'CN',
 'CN',
 'MCI',
 'CN',
 'Dem',
 'CN',
 'CN',
 'MCI',
 'CN',
 'CN',
 'CN',
 'CN',
 'Dem',
 'CN',
 'CN',
 'MCI',
 'MCI',
 'CN',
 'CN',
 'CN',
 'MCI',
 'CN',
 'MCI',
 'CN',
 'MCI',
 'CN',
 'CN',
 'Dem',
 'CN',
 'CN',
 'CN',
 'CN',
 'MCI',
 'MCI',
 'MCI',
 'CN',
 'MCI',
 'CN',
 'CN',
 'CN',
 'CN',
 'CN',
 'Dem',
 'MCI',
 'MCI',
 'MCI',
 'CN',
 'MCI',
 'MCI',
 'CN',
 'CN',
 'CN',
 'CN',
 'MCI',
 'CN',
 'CN',
 'MCI',
 'MCI',
 'CN',
 'CN',
 'CN',
 'Dem',
 'CN',
 'MCI',
 'Dem',
 'CN',
 'CN',
 'MCI',
 'CN',
 'CN',
 'CN',
 'MCI

In [9]:
del_col = [ 'EMAIL', 'summary_date',
                   'activity_class_5min', 'activity_met_1min',
                   'sleep_hr_5min', 'sleep_hypnogram_5min', 'sleep_rmssd_5min', 'timezone', 'sleep_total',
                   'CONVERT(activity_class_5min USING utf8)', 'CONVERT(activity_met_1min USING utf8)',
                   'CONVERT(sleep_hr_5min USING utf8)', 'CONVERT(sleep_hypnogram_5min USING utf8)',
                   'CONVERT(sleep_rmssd_5min USING utf8)']
inputs.drop(del_col, axis=1, inplace=True)
test.drop(del_col, axis=1, inplace=True)

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(inputs)
inputs = scaler.transform(inputs)

In [11]:
inputs.shape

(5180, 51)

In [12]:
len(arranged_labels)

148

In [13]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(arranged_labels)
le.classes_

array(['CN', 'Dem', 'MCI'], dtype='<U3')

In [14]:
arranged_labels = le.transform(arranged_labels)

In [15]:
arranged_labels

array([0, 2, 2, 0, 2, 0, 2, 0, 2, 0, 0, 2, 1, 0, 0, 0, 0, 2, 2, 0, 2, 0,
       0, 0, 2, 1, 0, 1, 0, 2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 1, 2, 0, 0,
       2, 0, 1, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 2, 2, 0, 0, 0, 2, 0, 2, 0,
       2, 0, 0, 1, 0, 0, 0, 0, 2, 2, 2, 0, 2, 0, 0, 0, 0, 0, 1, 2, 2, 2,
       0, 2, 2, 0, 0, 0, 0, 2, 0, 0, 2, 2, 0, 0, 0, 1, 0, 2, 1, 0, 0, 2,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [16]:
arranged_labels.shape

(148,)

In [17]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense, LSTM

x = inputs.reshape((148,35,51))

In [18]:
x.shape

(148, 35, 51)

In [19]:
y = arranged_labels

In [20]:
y.shape

(148,)

In [21]:
y

array([0, 2, 2, 0, 2, 0, 2, 0, 2, 0, 0, 2, 1, 0, 0, 0, 0, 2, 2, 0, 2, 0,
       0, 0, 2, 1, 0, 1, 0, 2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 1, 2, 0, 0,
       2, 0, 1, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 2, 2, 0, 0, 0, 2, 0, 2, 0,
       2, 0, 0, 1, 0, 0, 0, 0, 2, 2, 2, 0, 2, 0, 0, 0, 0, 0, 1, 2, 2, 2,
       0, 2, 2, 0, 0, 0, 0, 2, 0, 0, 2, 2, 0, 0, 0, 1, 0, 2, 1, 0, 0, 2,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [22]:
model = Sequential()
model.add(LSTM(24, activation = 'relu', input_shape=(35,51)))
model.add(Dense(12))
model.add(Dense(6))
model.add(Dense(3,activation='softmax'))

In [26]:
from keras.callbacks import EarlyStopping
epochs = 500
batch_size = 1

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
history = model.fit(
    x,
    y,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=1,
)

Epoch 1/500
118/118 [==============================] - 7s 58ms/step - loss: 0.0598 - sparse_categorical_accuracy: 0.9965 - val_loss: 3.6356 - val_sparse_categorical_accuracy: 0.5000
Epoch 2/500
118/118 [==============================] - 7s 56ms/step - loss: 0.0412 - sparse_categorical_accuracy: 0.9923 - val_loss: 3.9375 - val_sparse_categorical_accuracy: 0.5333
Epoch 3/500
118/118 [==============================] - 6s 54ms/step - loss: 0.0209 - sparse_categorical_accuracy: 0.9918 - val_loss: 5.2525 - val_sparse_categorical_accuracy: 0.5333
Epoch 4/500
118/118 [==============================] - 6s 48ms/step - loss: 0.4321 - sparse_categorical_accuracy: 0.9511 - val_loss: 3.1227 - val_sparse_categorical_accuracy: 0.5000
Epoch 5/500
118/118 [==============================] - 6s 51ms/step - loss: 0.2052 - sparse_categorical_accuracy: 0.9500 - val_loss: 2.5924 - val_sparse_categorical_accuracy: 0.5333
Epoch 6/500
118/118 [==============================] - 6s 53ms/step - loss: 0.0347 - spars

In [27]:
model = keras.models.load_model("best_model.h5")

test_loss, test_acc = model.evaluate(x, y)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

5/5 [==============================] - 0s 14ms/step - loss: 0.5516 - sparse_categorical_accuracy: 0.9054
Test accuracy 0.9054054021835327
Test loss 0.5515622496604919


In [28]:
yhat = model.predict(x)
print(yhat)


[[7.93986857e-01 1.07527361e-03 2.04937860e-01]
 [1.99493114e-03 1.57902147e-13 9.98005092e-01]
 [4.28916735e-07 9.80481252e-10 9.99999523e-01]
 [9.93495345e-01 1.95533547e-04 6.30909624e-03]
 [1.40216912e-03 5.12250971e-08 9.98597801e-01]
 [9.99630928e-01 4.10711946e-05 3.27906077e-04]
 [6.41977647e-03 1.03548437e-03 9.92544711e-01]
 [8.43498886e-01 1.70848507e-04 1.56330347e-01]
 [1.64667233e-06 1.08048264e-08 9.99998331e-01]
 [9.92394805e-01 1.29076798e-04 7.47608626e-03]
 [9.78762329e-01 6.78654807e-03 1.44510986e-02]
 [2.30053993e-04 2.63684607e-09 9.99769986e-01]
 [2.57570900e-06 9.99995112e-01 2.28712361e-06]
 [8.87443900e-01 8.10774392e-04 1.11745268e-01]
 [9.68052447e-01 3.06429160e-06 3.19444649e-02]
 [9.99980092e-01 2.37039804e-10 1.99591868e-05]
 [9.77961540e-01 3.15995247e-04 2.17225421e-02]
 [4.89192277e-01 1.30980370e-05 5.10794580e-01]
 [3.61741781e-02 6.23655978e-06 9.63819623e-01]
 [8.99619162e-01 7.29010499e-05 1.00307912e-01]
 [2.19637332e-06 4.70286754e-10 9.999978

In [29]:
yhat.shape

(148, 3)

ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets